In [ ]:
# import torch as th
# import torch.nn as nn
# import torch.nn.init as init
# import torch.nn.utils.prune as prune
# import torch
# import time
# #import pandas as pd
# import numpy as np
# import logging
# import csv
# from time import localtime, strftime
# import os
# import torch.nn.functional as F
# import torchvision
# import torchvision.transforms as transforms
# from itertools import zip_longest
# import matplotlib.pyplot as plt
# from matplotlib.ticker import PercentFormatter
# from torch.optim.lr_scheduler import MultiStepLR
# import os
# import random
# from torch.utils.tensorboard import SummaryWriter

# seed = 42
# th.manual_seed(seed)
# random.seed(seed)
# np.random.seed(seed)

# writer = SummaryWriter('results/resnet/resnet20/cifar104_1')
# device = 'cuda' if th.cuda.is_available() else 'cpu'

# class Network():

#     def weight_init(self, m):
#         if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
#             if self.a_type == 'relu':
#                 init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
#                 init.constant_(m.bias.data, 0)
#             elif self.a_type == 'leaky_relu':
#                 init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
#                 init.constant_(m.bias.data, 0)
#             elif self.a_type == 'tanh':
#                 g = init.calculate_gain(self.a_type)
#                 init.xavier_uniform_(m.weight.data, gain=g)
#                 init.constant_(m.bias.data, 0)
#             elif self.a_type == 'sigmoid':
#                 g = init.calculate_gain(self.a_type)
#                 init.xavier_uniform_(m.weight.data, gain=g)
#                 init.constant_(m.bias.data, 0)
#             else:
#                 raise
#                 return NotImplemented

# class PruningMethod():

#     def prune_filters(self,indices):
#       conv_layer=0


#       for layer_name, layer_module in self.named_modules():

#         if(isinstance(layer_module, th.nn.Conv2d)  and layer_name!='conv1'):

#           if(layer_name.find('conv1')!=-1):
#             in_channels=[i for i in range(layer_module.weight.shape[1])]
#             out_channels=indices[conv_layer]
#             layer_module.weight = th.nn.Parameter( th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])).to('cuda'))



#           if(layer_name.find('conv2')!=-1):
#              in_channels=indices[conv_layer]
#              out_channels=[i for i in range(layer_module.weight.shape[0])]
#              layer_module.weight = th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[:,in_channels])).to('cuda'))
#              conv_layer+=1


#           layer_module.in_channels=len(in_channels)
#           layer_module.out_channels=len(out_channels)


#         if (isinstance(layer_module, th.nn.BatchNorm2d) and layer_name!='bn1' and layer_name.find('bn1')!=-1):
#             out_channels=indices[conv_layer]


#             layer_module.weight=th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])).to('cuda'))
#             layer_module.bias=th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.bias.data.cpu().numpy()[out_channels])).to('cuda'))



#             layer_module.running_mean= th.from_numpy(layer_module.running_mean.cpu().numpy()[out_channels]).to('cuda')
#             layer_module.running_var=th.from_numpy(layer_module.running_var.cpu().numpy()[out_channels]).to('cuda')



#             layer_module.num_features= len(out_channels)

#         if isinstance(layer_module, nn.Linear):

#             break


#     def get_indices_topk(self,layer_bounds,layer_num,prune_limit,prune_value):

#       i=layer_num
#       indices=prune_value[i]

#       p=len(layer_bounds)
#       if (p-indices)<prune_limit:
#          prune_value[i]=p-prune_limit
#          indices=prune_value[i]

#       k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[:indices]
#       return k

#     def get_indices_bottomk(self,layer_bounds,i,prune_limit):

#       k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[-prune_limit:]
#       return k

# norm_mean, norm_var = 0.0, 1.0


# def conv3x3(in_planes, out_planes, stride=1):
#     return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
#                      padding=1, bias=False)


# class LambdaLayer(nn.Module):
#     def __init__(self, lambd):
#         super(LambdaLayer, self).__init__()
#         self.lambd = lambd

#     def forward(self, x):
#         return self.lambd(x)


# class ResBasicBlock(nn.Module,Network,PruningMethod):
#     expansion = 1

#     def __init__(self, inplanes, planes, stride=1):
#         super(ResBasicBlock, self).__init__()
#         self.inplanes = inplanes
#         self.planes = planes
#         self.conv1 = conv3x3(inplanes, planes, stride)
#         self.bn1 = nn.BatchNorm2d(planes)
#         self.relu1 = nn.ReLU(inplace=True)

#         self.conv2 = conv3x3(planes, planes)
#         self.bn2 = nn.BatchNorm2d(planes)
#         self.relu2 = nn.ReLU(inplace=True)
#         self.stride = stride
#         self.shortcut = nn.Sequential()
#         if stride != 1 or inplanes != planes:
#             self.shortcut = LambdaLayer(
#                 lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes // 4, planes-inplanes-(planes//4)), "constant", 0))

#     def forward(self, x):
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu1(out)

#         out = self.conv2(out)
#         out = self.bn2(out)

#         out += self.shortcut(x)
#         out = self.relu2(out)

#         return out


# class ResNet(nn.Module,Network,PruningMethod):
#     def __init__(self, block, num_layers, covcfg,num_classes=10):
#         super(ResNet, self).__init__()
#         assert (num_layers - 2) % 6 == 0, 'depth should be 6n+2'
#         n = (num_layers - 2) // 6
#         self.covcfg = covcfg
#         self.num_layers = num_layers

#         self.inplanes = 16
#         self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)


#         self.bn1 = nn.BatchNorm2d(self.inplanes)
#         self.relu = nn.ReLU(inplace=True)

#         self.layer1 = self._make_layer(1,block, 16, blocks=n, stride=1)
#         self.layer2 = self._make_layer(2,block, 32, blocks=n, stride=2)
#         self.layer3 = self._make_layer(3,block, 64, blocks=n, stride=2)
#         self.avgpool = nn.AdaptiveAvgPool2d(1)

#         if num_layers == 110:
#             self.linear = nn.Linear(64 * block.expansion, num_classes)
#         else:
#             self.fc = nn.Linear(64 * block.expansion, num_classes)

#         self.initialize()
#         self.layer_name_num={}
#         self.pruned_filters={}
#         self.remaining_filters={}

#         self.remaining_filters_each_epoch=[]

#     def initialize(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#     def _make_layer(self,a, block, planes, blocks, stride):
#         layers = []

#         layers.append(block(self.inplanes, planes, stride))

#         self.inplanes = planes * block.expansion
#         for i in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)

#     def forward(self, x):

#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)

#         if self.num_layers == 110:
#             x = self.linear(x)
#         else:
#             x = self.fc(x)

#         return x


# def resnet_56():
#     cov_cfg = [(3 * i + 2) for i in range(9 * 3 * 2 + 1)]
#     return ResNet(ResBasicBlock, 56, cov_cfg)
# def resnet_110():
#     cov_cfg = [(3 * i + 2) for i in range(18 * 3 * 2 + 1)]
#     return ResNet(ResBasicBlock, 110, cov_cfg)
# def resnet_104():
#     cov_cfg = [(3 * i + 2) for i in range(17 * 3 * 2 + 1)]
#     return ResNet(ResBasicBlock, 104, cov_cfg)
# def resnet_32():
#     cov_cfg = [16, 16, 32, 64]
#     return ResNet(ResBasicBlock, 32, cov_cfg)
# def resnet_20():
#     cov_cfg = [(3 * i + 2) for i in range(3 * 3 * 2 + 1)]
#     return ResNet(ResBasicBlock, 20, cov_cfg)
# def resnet_34():
#     cov_cfg = [(3 * i + 2) for i in range(3 * 3 + 4 * 3 + 6 * 3 + 3 * 3 + 1)]
#     return ResNet(ResBasicBlock, 34, cov_cfg)

# th.manual_seed(seed)
# th.cuda.manual_seed(seed)
# th.cuda.manual_seed_all(seed)
# th.backends.cudnn.deterministic = True
# N = 1

# batch_size_tr = 128
# batch_size_te = 128

# epochs = 164
# lr=0.1
# milestones_array=[82,123]
# lamda=0.000001
# weight_decay=1e-4
# momentum=0.9

# prune_limits=[6]*3*3
# prune_value=[1]*3+[2]*3+[4]*3

# total_layers=20
# total_convs=9
# total_blocks=3

# # th.cuda.set_device(0)
# gpu = th.cuda.is_available()

# # model=resnet_56()

# if not gpu:
#     print('qqqq')
# else:
#     transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])

#     transform_test = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     ])

#     trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
#     trainloader = th.utils.data.DataLoader(trainset, batch_size=batch_size_tr, shuffle=True, num_workers=2)

#     testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
#     testloader = th.utils.data.DataLoader(testset, batch_size=batch_size_te, shuffle=True, num_workers=2)

# decision_count=th.ones((total_convs))

# short=False
# tr_size = 50000
# te_size=10000


# activation = 'relu'

# if gpu:
#     model=resnet_20().cuda()
# else:
#     model=resnet_20()




In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (shortcut): Sequential()
    )
    (1): ResBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Co

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = th.optim.SGD(model.parameters(), lr=lr,momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = MultiStepLR(optimizer, milestones=milestones_array, gamma=0.1)

def evaluate(model, valid_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valid_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = total_loss / len(valid_loader)
    accuracy = 100 * correct / total
    return loss, accuracy


def custom_loss(outputs, labels, model, criterion, lambda_l1):
    l1_norm = 0
    for param in model.parameters():
        l1_norm += torch.sum(torch.abs(param))
    # Cross-entropy loss
    ce_loss = criterion(outputs, labels)
    # Total loss with L1 regularization
    total_loss = ce_loss - lambda_l1 * l1_norm
    return total_loss

ans1='f'
if(ans1=='t'):
  checkpoint = th.load('resnet56_base.pth')
  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  scheduler.load_state_dict(checkpoint['scheduler'])
  epoch_train_acc = checkpoint['train_acc']
  print(epoch_train_acc,'.......')
  epoch_test_acc = checkpoint['test_acc']
  print('model loaded')

elif(ans1=='f'):

    best_train_acc=0
    best_test_acc=0

    for n in range(1):

        mi_iteration=0
        for epoch in range(epochs):

          train_acc=[]
          for batch_num, (inputs, targets) in enumerate(trainloader):

            inputs = inputs.cuda()
            targets = targets.cuda()

            model.train()

            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            with th.no_grad():
              y_hat = th.argmax(output, 1)
              score = th.eq(y_hat, targets).sum()
              train_acc.append(score.item())


          with th.no_grad():
            epoch_train_acc=  (sum(train_acc)*100)/tr_size
            test_acc=[]
            model.eval()
            for batch_nums, (inputs2, targets2) in enumerate(testloader):
                if(batch_nums==3 and short):
                    break

                inputs2, targets2 = inputs2.cuda(), targets2.cuda()
                output=model(inputs2)
                y_hat = th.argmax(output, 1)
                score = th.eq(y_hat, targets2).sum()
                test_acc.append(score.item())

            epoch_test_acc= (sum(test_acc)*100)/te_size
          writer.add_scalar('Training Loss', loss.item(), epoch)
          writer.add_scalar('Training Accuracy', epoch_train_acc, epoch)
          writer.add_scalar('Test Accuracy', epoch_test_acc, epoch)

          print('\n---------------Epoch number: {}'.format(epoch),
                  '---Train accuracy: {}'.format(epoch_train_acc),
                  '----Test accuracy: {}'.format(epoch_test_acc),'--------------')
          scheduler.step()
        #   print(optimizer.param_groups[0]['lr'])
else:
   print('wrong ans entered')
   import sys
   sys.exit()



NameError: name 'trainloader' is not defined

In [ ]:

a=[]
for layer_name, layer_module in model.named_modules():
  if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
    a.append(layer_module)

# print("Training accuracy of baseline model is ",epoch_train_acc,'.......')
# print("Testing accuracy of baseline model is ",epoch_test_acc,'.......')


# # ended_epoch=0
# best_train_acc=epoch_train_acc
# best_test_acc=epoch_test_acc

# epoch=0
# writer.add_scalar('Baseline Accuracy/Train', epoch_train_acc, epoch)
# writer.add_scalar('Baseline Accuracy/Test', epoch_test_acc, epoch)


with th.no_grad():

      #_______________________COMPUTE L1NORM____________________________________
    l1norm=[]
    l_num=0
    for layer_name, layer_module in model.named_modules():

        if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
            temp=[]
            filter_weight=layer_module.weight.clone()

            for k in range(filter_weight.size()[0]):
                temp.append(float("{:.6f}".format((filter_weight[k,:,:,:]).norm(1).item())))

            l1norm.append(temp)
            l_num+=1

    layer_bounds1=l1norm
#______Selecting__filters__to__regularize_____

    inc_indices=[]
    for i in range(len(layer_bounds1)):
        imp_indices=model.get_indices_bottomk(layer_bounds1[i],i,prune_limits[i])
        inc_indices.append(imp_indices)



    unimp_indices=[]
    dec_indices=[]
    for i in range(len(layer_bounds1)):
        temp=[]
        temp=model.get_indices_topk(layer_bounds1[i],i,prune_limits[i],prune_value)
        unimp_indices.append(temp[:])
        temp.extend(inc_indices[i])
        dec_indices.append(temp)

    # print('selected  UNIMP indices ',unimp_indices)

    remaining_indices=[]
    for i in range(total_convs):
      temp=[]
      for j in range(a[i].weight.shape[0]):
        if (j not in unimp_indices[i]):
          temp.extend([j])
      remaining_indices.append(temp)
continue_pruning=True
with th.no_grad():

    if(continue_pruning==True):
        model.prune_filters(remaining_indices)
        print(model)

ended_epoch=0
best_train_acc=0
best_test_acc=0


decision=True
best_test_acc= 0.0
prunes=1


In [ ]:

while(continue_pruning==True):

  if(continue_pruning==True):


    if(th.sum(decision_count)==0):
          continue_pruning=False
    with th.no_grad():

      #_______________________COMPUTE L1NORM____________________________________
      l1norm=[]
      l_num=0
      for layer_name, layer_module in model.named_modules():

          if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
              temp=[]
              filter_weight=layer_module.weight.clone()

              for k in range(filter_weight.size()[0]):
                temp.append(float("{:.6f}".format((filter_weight[k,:,:,:]).norm(1).item())))

              l1norm.append(temp)
              l_num+=1

      layer_bounds1=l1norm

#______Selecting__filters__to__regularize_____

    inc_indices=[]
    for i in range(len(layer_bounds1)):
        imp_indices=model.get_indices_bottomk(layer_bounds1[i],i,prune_limits[i])
        inc_indices.append(imp_indices)

    unimp_indices=[]
    dec_indices=[]
    for i in range(len(layer_bounds1)):
        temp=[]
        temp=model.get_indices_topk(layer_bounds1[i],i,prune_limits[i],prune_value)
        unimp_indices.append(temp[:])
        temp.extend(inc_indices[i])
        dec_indices.append(temp)

    # print('selected  UNIMP indices ',unimp_indices)

    remaining_indices=[]
    for i in range(total_convs):
      temp=[]
      for j in range(a[i].weight.shape[0]):
        if (j not in unimp_indices[i]):
          temp.extend([j])
      remaining_indices.append(temp)

    if(continue_pruning==False):
       lamda=0
#______________________Custom_Regularize the model___________________________
    # if(continue_pruning==True):
    #    optimizer = th.optim.SGD(model.parameters(), lr=optim_lr,momentum=0.9)
    #    scheduler = MultiStepLR(optimizer, milestones=milestones_array, gamma=0.1)
    optimizer = th.optim.SGD(model.parameters(), lr=lr,momentum=0.9, weight_decay=1e-4,nesterov=True)
    scheduler = MultiStepLR(optimizer, milestones=milestones_array, gamma=0.1)

    epoch=0
    best_test_acc= 0.0
    for epoch in range(epochs):

        i=0
        start_time = time.time()
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch_idx, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            if batch_idx==0:
                print("Loss cross-entropy = ",criterion(outputs,labels))
                l1_reg = sum(torch.sum(torch.abs(param)) for param in model.parameters())
                print("L1 NORM = ", l1_reg)
                print("New loss  = ",loss-(lamda*l1_reg))
            loss = criterion(outputs, labels)
            l1_reg = sum(torch.sum(torch.abs(param)) for param in model.parameters())
            loss=loss-(lamda*l1_reg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        scheduler.step()
        epoch_loss = running_loss / len(trainloader.dataset)
        epoch_acc = 100 * correct / total

        valid_loss, valid_acc = evaluate(model, testloader, criterion)
        end_time=time.time()
        total_time=end_time-start_time

        print('\nEpoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.2f}%, Valid Loss: {:.4f}, Valid Accuracy: {:.2f}%, Time: {:.2f}s'.format(
            epoch + 1, epochs, epoch_loss, epoch_acc, valid_loss, valid_acc,total_time))


    writer.add_scalar(f'Pruning_{prunes}/Train Accuracy', epoch_acc, prunes)
    writer.add_scalar(f'Pruning_{prunes}/Test Accuracy', valid_acc, prunes)

    # Log the model structure (you can add it once or multiple times if the structure changes)
    writer.add_text(f'Pruning_{prunes}/Model Structure', str(model), prunes)

    with th.no_grad():
      #_______________________COMPUTE L1NORM____________________________________

      l1norm=[]
      l_num=0
      for layer_name, layer_module in model.named_modules():

          if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
              temp=[]
              filter_weight=layer_module.weight.clone()
              for k in range(filter_weight.size()[0]):
                      temp.append(float("{:.6f}".format((filter_weight[k,:,:,:]).norm(1).item())))
              l1norm.append(temp)
              l_num+=1

      layer_bounds1=l1norm

    with th.no_grad():

      if(continue_pruning==True):
        model.prune_filters(remaining_indices)
        print(model)
      else:
        break

      #_________________________PRUNING_EACH_CONV_LAYER__________________________
      for i in range(len(layer_bounds1)):
          if(a[i].weight.shape[0]<= prune_limits[i]):
            decision_count[:]=0
            break


      prunes+=1



In [ ]:
import torch as th
import torch.nn as nn
import torch.nn.init as init
import torch.nn.utils.prune as prune
import torch
import time
#import pandas as pd
import numpy as np
import logging
import csv
from time import localtime, strftime
import os
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from itertools import zip_longest
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from torch.optim.lr_scheduler import MultiStepLR
import os
import random
import math
from torch.utils.tensorboard import SummaryWriter

seed = 42
th.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

writer = SummaryWriter('results/cifar-10/base')
device = 'cuda' if th.cuda.is_available() else 'cpu'

class Network():

    def weight_init(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            if self.a_type == 'relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'leaky_relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'tanh':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'sigmoid':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            else:
                raise
                return NotImplemented

class PruningMethod():

    def prune_filters(self,indices):
      conv_layer=0


      for layer_name, layer_module in self.named_modules():

        if(isinstance(layer_module, th.nn.Conv2d)  and layer_name!='conv1'):

          if(layer_name.find('conv1')!=-1):
            in_channels=[i for i in range(layer_module.weight.shape[1])]
            out_channels=indices[conv_layer]
            layer_module.weight = th.nn.Parameter( th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])).to('cuda'))



          if(layer_name.find('conv2')!=-1):
             in_channels=indices[conv_layer]
             out_channels=[i for i in range(layer_module.weight.shape[0])]
             layer_module.weight = th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[:,in_channels])).to('cuda'))
             conv_layer+=1


          layer_module.in_channels=len(in_channels)
          layer_module.out_channels=len(out_channels)


        if (isinstance(layer_module, th.nn.BatchNorm2d) and layer_name!='bn1' and layer_name.find('bn1')!=-1):
            out_channels=indices[conv_layer]


            layer_module.weight=th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.weight.data.cpu().numpy()[out_channels])).to('cuda'))
            layer_module.bias=th.nn.Parameter(th.FloatTensor(th.from_numpy(layer_module.bias.data.cpu().numpy()[out_channels])).to('cuda'))



            layer_module.running_mean= th.from_numpy(layer_module.running_mean.cpu().numpy()[out_channels]).to('cuda')
            layer_module.running_var=th.from_numpy(layer_module.running_var.cpu().numpy()[out_channels]).to('cuda')



            layer_module.num_features= len(out_channels)

        if isinstance(layer_module, nn.Linear):

            break


    def get_indices_topk(self,layer_bounds,layer_num,prune_limit,prune_value):

      i=layer_num
      indices=prune_value[i]

      p=len(layer_bounds)
      if (p-indices)<prune_limit:
         prune_value[i]=p-prune_limit
         indices=prune_value[i]

      k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[:indices]
      return k

    def get_indices_bottomk(self,layer_bounds,i,prune_limit):

      k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[-prune_limit:]
      return k

norm_mean, norm_var = 0.0, 1.0


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class ResBasicBlock(nn.Module,Network,PruningMethod):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1):
        super(ResBasicBlock, self).__init__()
        self.inplanes = inplanes
        self.planes = planes
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.stride = stride
        self.shortcut = nn.Sequential()
        if stride != 1 or inplanes != planes:
            self.shortcut = LambdaLayer(
                lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes // 4, planes-inplanes-(planes//4)), "constant", 0))

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(x)
        out = self.relu2(out)

        return out


class ResNet(nn.Module,Network,PruningMethod):
    def __init__(self, block, num_layers, covcfg,num_classes=10):
        super(ResNet, self).__init__()
        assert (num_layers - 2) % 6 == 0, 'depth should be 6n+2'
        n = (num_layers - 2) // 6
        self.covcfg = covcfg
        self.num_layers = num_layers

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)


        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(1,block, 16, blocks=n, stride=1)
        self.layer2 = self._make_layer(2,block, 32, blocks=n, stride=2)
        self.layer3 = self._make_layer(3,block, 64, blocks=n, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        if num_layers == 110:
            self.linear = nn.Linear(64 * block.expansion, num_classes)
        else:
            self.fc = nn.Linear(64 * block.expansion, num_classes)

        self.initialize()
        self.layer_name_num={}
        self.pruned_filters={}
        self.remaining_filters={}

        self.remaining_filters_each_epoch=[]

    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self,a, block, planes, blocks, stride):
        layers = []

        layers.append(block(self.inplanes, planes, stride))

        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        if self.num_layers == 110:
            x = self.linear(x)
        else:
            x = self.fc(x)

        return x


def resnet_56():
    cov_cfg = [(3 * i + 2) for i in range(9 * 3 * 2 + 1)]
    return ResNet(ResBasicBlock, 56, cov_cfg)
def resnet_110():
    cov_cfg = [(3 * i + 2) for i in range(18 * 3 * 2 + 1)]
    return ResNet(ResBasicBlock, 110, cov_cfg)
def resnet_104():
    cov_cfg = [(3 * i + 2) for i in range(17 * 3 * 2 + 1)]
    return ResNet(ResBasicBlock, 104, cov_cfg)
def resnet_32():
    cov_cfg = [16, 16, 32, 64]
    return ResNet(ResBasicBlock, 32, cov_cfg)
def resnet_20():
    cov_cfg = [(3 * i + 2) for i in range(3 * 3 * 2 + 1)]
    return ResNet(ResBasicBlock, 20, cov_cfg)
def resnet_34():
    cov_cfg = [(3 * i + 2) for i in range(3 * 3 + 4 * 3 + 6 * 3 + 3 * 3 + 1)]
    return ResNet(ResBasicBlock, 34, cov_cfg)

th.manual_seed(seed)
th.cuda.manual_seed(seed)
th.cuda.manual_seed_all(seed)
th.backends.cudnn.deterministic = True
N = 1

batch_size_tr = 128
batch_size_te = 128

epochs = 164
lr=0.1
milestones_array=[82,123]
lamda=0.000001
weight_decay=1e-4
momentum=0.9

prune_limits=[6]*3*3
prune_value=[1]*3+[2]*3+[4]*3

total_layers=20
total_convs=9
total_blocks=3

# th.cuda.set_device(0)
gpu = th.cuda.is_available()

# model=resnet_56()


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

trainset = torchvision.datasets.CIFAR10(root='../../data', train=True, download=True, transform=transform_train)
trainloader = th.utils.data.DataLoader(trainset, batch_size=batch_size_tr, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../../data', train=False, download=True, transform=transform_test)
testloader = th.utils.data.DataLoader(testset, batch_size=batch_size_te, shuffle=True, num_workers=2)

decision_count=th.ones((total_convs))

short=False
tr_size = 50000
te_size=10000


activation = 'relu'

if gpu:
    model=resnet_20().cuda()
else:
    model=resnet_20()


criterion = nn.CrossEntropyLoss()

optimizer = th.optim.SGD(model.parameters(), lr=lr,momentum=0.9, weight_decay=1e-4,nesterov=True)
scheduler = MultiStepLR(optimizer, milestones=milestones_array, gamma=0.1)

def evaluate(model, valid_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valid_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = total_loss / len(valid_loader)
    accuracy = 100 * correct / total
    return loss, accuracy


def custom_loss(outputs, labels, model, criterion, lambda_l1):
    l1_norm = 0
    for param in model.parameters():
        l1_norm += torch.sum(torch.abs(param))
    ce_loss = criterion(outputs, labels)
    total_loss = ce_loss - lambda_l1 * l1_norm
    return total_loss

def train(model, train_loader, test_loader, epochs, lr, optimizer, criterion):
    best_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        train_loss = 0.0

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            lambda_l1 = 0.01
            loss = custom_loss(outputs, targets, model, criterion, lambda_l1)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()

        avg_train_loss = train_loss / train_total
        train_accuracy = 100. * train_correct / train_total

        test_loss, test_accuracy = evaluate(model, test_loader, criterion)

        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, '
              f'Test Accuracy: {test_accuracy:.2f}%')

        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_accuracy, epoch)
        writer.add_scalar('Loss/Test', test_loss, epoch)
        writer.add_scalar('Accuracy/Test', test_accuracy, epoch)

        scheduler.step()

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model



a=[]
for layer_name, layer_module in model.named_modules():
  if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
    a.append(layer_module)

def prune(model):

    with th.no_grad():

        l1norm=[]
        l_num=0
        for layer_name, layer_module in model.named_modules():

            if(isinstance(layer_module, th.nn.Conv2d) and layer_name!='conv1' and layer_name.find('conv1')!=-1):
                temp=[]
                filter_weight=layer_module.weight.clone()

                for k in range(filter_weight.size()[0]):
                    temp.append(float("{:.6f}".format((filter_weight[k,:,:,:]).norm(1).item())))

                l1norm.append(temp)
                l_num+=1

        layer_bounds1=l1norm

        inc_indices=[]
        for i in range(len(layer_bounds1)):
            imp_indices=model.get_indices_bottomk(layer_bounds1[i],i,prune_limits[i])
            inc_indices.append(imp_indices)


        unimp_indices=[]
        dec_indices=[]
        for i in range(len(layer_bounds1)):
            temp=[]
            temp=model.get_indices_topk(layer_bounds1[i],i,prune_limits[i],prune_value)
            unimp_indices.append(temp[:])
            temp.extend(inc_indices[i])
            dec_indices.append(temp)


        remaining_indices=[]
        for i in range(total_convs):
            temp=[]
        for j in range(a[i].weight.shape[0]):
            if (j not in unimp_indices[i]):
              temp.extend([j])
        remaining_indices.append(temp)
    continue_pruning=True
    with th.no_grad():

        if(continue_pruning==True):
            model.prune_filters(remaining_indices)

    return model

def complete(model):
    while True:
        model = prune(model)
        print(model)
        train(model, trainloader, testloader, epochs, lr, optimizer, criterion)
    return model

# print(model)
# model = complete(model)


# ended_epoch=0
# best_train_acc=0
# best_test_acc=0


# decision=True
# best_test_acc= 0.0
# prunes=1


100%|██████████| 170M/170M [00:03<00:00, 46.7MB/s]


Extracting ../../data/cifar-10-python.tar.gz to ../../data
Files already downloaded and verified


In [ ]:

class Network():
    def __init__(self, model):
        self.model = model
        self.layer_outputs = {}

    def get_layer_output(self, x, labels, layer_name):
        self.model.eval()

        def hook(module, input, output):
          self.layer_outputs[layer_name] = output

        hook_handle = None
        for name, module in self.model.named_modules():
            if name == layer_name:
                hook_handle = module.register_forward_hook(hook)
                break

        with torch.no_grad():
          self.model(x)

        hook_handle.remove()

        if layer_name in self.layer_outputs:
            output = self.layer_outputs[layer_name]
        else:
            raise ValueError(f"No output captured for layer: {layer_name}")

        labels_expanded = labels[:, None] == labels[None, :]
        label_matrix = torch.where(labels_expanded, 1.0, math.sqrt(2)).to(device)

        return label_matrix, output

    def dist_mat(self, x):
        if len(x.shape) == 4:
            x = x.view(x.shape[0], -1)

        dist = torch.cdist(x, x, p=2)
        return dist

    def diff_and_forb(self, data):

        num_images = data.shape[0]
        num_filters = data.shape[1]

        diff_matrix = data.unsqueeze(0) - data.unsqueeze(1)
        results = torch.norm(diff_matrix, dim=(3, 4))

        return results

    def compute_gaussian_kernel(self, frobenius_norm_matrices, sigma):

        sigma = torch.tensor(sigma, device=frobenius_norm_matrices.device)
        gaussian_kernels = torch.exp(-frobenius_norm_matrices ** 2 / (2 * sigma ** 2))
        gaussian_kernels = torch.exp(-frobenius_norm_matrices ** 2 / (2 * sigma ** 2))

        return gaussian_kernels

    def normalize_matrix(self, matrix):
        assert matrix.shape[0] == matrix.shape[1], "Input matrix must be square"

        matrix = matrix.cpu()
        normalized_matrix = np.zeros_like(matrix.numpy(), dtype=np.float64)

        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if i == j:
                    normalized_matrix[i, j] = 1.0
                else:
                    normalized_matrix[i, j] = matrix[i, j] / np.sqrt(matrix[i, i] * matrix[j, j])

        return torch.tensor(normalized_matrix, device=matrix.device)

    def hadamard_mult(self, data, labels):
        if isinstance(labels, np.ndarray):
            labels = torch.from_numpy(labels)
        if isinstance(data, np.ndarray):
            data = torch.from_numpy(data)
        broadcasted_labels = labels.unsqueeze(-1).expand_as(data)
        result = data * broadcasted_labels
        return result

    def entropy(self, result):
        entropies = np.zeros(result.shape[-1])
        epsilon = torch.finfo(float).eps

        for i in range(result.shape[-1]):
            eigenvalues = np.linalg.eigvals(result[:, :, i])
            eigenvalues = np.abs(eigenvalues)
            eigenvalues[eigenvalues == 0] = epsilon

            entropy = -np.sum(eigenvalues * np.log2(eigenvalues))

            entropies[i] = entropy

        return entropies

    def dist_mat(self, x):
        if len(x.shape) == 4:
            x = x.view(x.shape[0], -1)

        x_squared = (x ** 2).sum(dim=1, keepdim=True)  # Shape: (N, 1)
        distances = torch.sqrt(x_squared - 2 * (x @ x.T) + x_squared.T)
        return distances


    def cal_mi(self, x, labels, conv_layers):
        layer_entropies = {}

        for layer in conv_layers:

          layer_name = layer

          label_matrix, layer_output = self.get_layer_output(x, labels, layer_name)

          frobenius_norm = self.diff_and_forb(layer_output)

          gaussian_kernel = self.compute_gaussian_kernel(frobenius_norm, 2)

          normalized_gauss = self.normalize_matrix(gaussian_kernel)

          normalized_labels = self.normalize_matrix(label_matrix)

          hadamard_mult = self.hadamard_mult(normalized_gauss, normalized_labels)

          entropy = self.entropy(hadamard_mult)

          label_matrix = self.normalize_matrix(label_matrix)

          layer_entropies[layer_name] = entropy

        return layer_entropies


def calculate_top_filters(base_model, train_loader, conv_layers):
    batch_size = 128
    network = Network(base_model)

    layer_votes_batch = {}
    layer_aggregated_votes = {}

    for conv in conv_layers:
        layer_votes_batch[conv] = []

    print(conv_layers)

    for batch_images, batch_labels in train_loader:

        batch_images = batch_images.to(device)
        batch_labels = batch_labels.to(device)
        layer_votes = network.cal_mi(batch_images, batch_labels, conv_layers)

        for layer in conv_layers:
            layer_votes_batch[layer].append(layer_votes[layer])


    for layer in conv_layers:
        layer_aggregated_votes[layer] = np.mean(layer_votes_batch[layer], axis=0)

    return layer_aggregated_votes

def sort(layer_aggregated_votes, conv_layers, k):
    layer_result = {}
    for layer in conv_layers:
        agg_votes = layer_aggregated_votes[layer]
        k_indices=int((agg_votes.size) * k)
        flat_indices = np.argsort(agg_votes.ravel())[-k_indices:]
        indices = np.unravel_index(flat_indices, agg_votes.shape)
        result = np.zeros_like(agg_votes)
        result[indices] = 1
        layer_result[layer] = result

    return layer_result



def get_important_filters(model, train_loader):
    conv_layers = []
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d):  # Identify all convolutional layers
            conv_layers.append(name)

    most_important_filters = calculate_top_filters(model, train_loader, conv_layers)
    sorted_filters = sort(most_important_filters, conv_layers, 0.5)
    return sorted_filters, conv_layers

In [ ]:
filter_importance, conv_layers = get_important_filters(model, trainloader)
for name, layer in model.named_modules():
    if "conv" in name:
        print(name)
        print(layer)

['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.2.conv1', 'layer3.2.conv2']


In [ ]:
layer_idx = list(model._modules).index('conv2')
intermediate_model = nn.Sequential(*list(model.children())[:layer_idx+1]).to(device)
# output = intermediate_model(x)

ValueError: 'conv2' is not in list

In [ ]:
def train_model_resnet(model, train_loader, test_loader, epochs=250, lr=0.1, lr_drop=20, base=False, selected_filters=None, conv_layers=None):
    global pruning
    best_accuracy = 0
    best_model = None
    pruning += 1

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_drop, gamma=0.5)

    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            if base:
                loss = criterion(outputs, targets)
            else:
                loss = custom_loss(outputs, targets, model, criterion, params["lambda_l1"])

            loss.backward()

            # Gradient manipulation for selected filters
            for layer_name in conv_layers:
                # Accessing specific layer from ResNet
                module_hierarchy = layer_name.split('.')
                conv_layer = model
                for module_name in module_hierarchy:
                    conv_layer = getattr(conv_layer, module_name)

                filters_state = list(selected_filters[layer_name])
                filter_mask = torch.tensor(filters_state, dtype=torch.float32).view(-1, 1, 1, 1).to(device)

                with torch.no_grad():
                    conv_layer.weight.grad *= filter_mask

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        train_accuracy = 100. * correct / total
        avg_train_loss = train_loss / total

        test_loss, test_accuracy = test_model(model, test_loader, criterion)

        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        writer.add_scalar(f'Loss/Train {pruning}Prune', train_loss, epoch)
        writer.add_scalar(f'Accuracy/Train {pruning}Prune', train_accuracy, epoch)
        writer.add_scalar(f'Loss/Test {pruning}Prune', test_loss, epoch)
        writer.add_scalar(f'Accuracy/Test {pruning}Prune', test_accuracy, epoch)

        scheduler.step()

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model = deepcopy(model)

    return best_model

In [ ]:
intermediate_model

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [ ]:
for name, layer in model.named_modules():
  if isinstance(layer, nn.Conv2d):
    print(name)
    # print(name)
    # print(type(layer))

conv1
layer1.0.conv1
layer1.0.conv2
layer1.1.conv1
layer1.1.conv2
layer1.2.conv1
layer1.2.conv2
layer2.0.conv1
layer2.0.conv2
layer2.1.conv1
layer2.1.conv2
layer2.2.conv1
layer2.2.conv2
layer3.0.conv1
layer3.0.conv2
layer3.1.conv1
layer3.1.conv2
layer3.2.conv1
layer3.2.conv2


In [ ]:
model

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (shortcut): Sequential()
    )
    (1): ResBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Co